In [71]:
! pip install pyspark

In [72]:
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from pyspark.sql.functions import col, sum
import pyspark.sql.functions as F

In [73]:
spark = SparkSession.builder.getOrCreate()

In [74]:
df=spark.read.csv("/kaggle/input/datasets-for-pyspark-project/airports.csv",header=True)

In [75]:
df.show(10)

+---+--------------------+----------+------------+----+---+---+
|faa|                name|       lat|         lon| alt| tz|dst|
+---+--------------------+----------+------------+----+---+---+
|04G|   Lansdowne Airport|41.1304722| -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|32.4605722| -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|41.9893408| -88.1012428| 801| -6|  A|
|06N|     Randall Airport| 41.431912| -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|31.0744722| -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|36.3712222| -82.1734167|1593| -4|  A|
|0G6|Williams County A...|41.4673056| -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|42.8835647| -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|39.7948244| -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|48.0538086|-122.8106436| 108| -8|  A|
+---+--------------------+----------+------------+----+---+---+
only showing top 10 rows



In [76]:
print("Columns:", df.columns)

Columns: ['faa', 'name', 'lat', 'lon', 'alt', 'tz', 'dst']


In [77]:
null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])
null_counts.show()

+---+----+---+---+---+---+---+
|faa|name|lat|lon|alt| tz|dst|
+---+----+---+---+---+---+---+
|  0|   0|  0|  0|  0|  0|  0|
+---+----+---+---+---+---+---+



In [78]:
df2 = spark.read.csv('/kaggle/input/datasets-for-pyspark-project/flights_small.csv', header=True)
df2.show(10)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [79]:
print(df2.columns)

['year', 'month', 'day', 'dep_time', 'dep_delay', 'arr_time', 'arr_delay', 'carrier', 'tailnum', 'flight', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute']


In [80]:
null_counts_2 = df2.select([sum(col(c).isNull().cast("int")).alias(c) for c in df2.columns])
null_counts_2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|   0|    0|  0|       0|        0|       0|        0|      0|      0|     0|     0|   0|       0|       0|   0|     0|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+



In [81]:
df2=df2.withColumn("duration_hrs",df2.air_time/60)
df2.show(5)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

In [82]:
df2.describe().show()

+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-------+-------+-----------------+------+-----+------------------+-----------------+------------------+-----------------+------------------+
|summary|  year|             month|              day|          dep_time|         dep_delay|          arr_time|         arr_delay|carrier|tailnum|           flight|origin| dest|          air_time|         distance|              hour|           minute|      duration_hrs|
+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-------+-------+-----------------+------+-----+------------------+-----------------+------------------+-----------------+------------------+
|  count| 10000|             10000|            10000|             10000|             10000|             10000|             10000|  10000|  10000|            10000| 10000|10000|             1

In [83]:
long_flight=df2.filter("distance>1000")
long_flight.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|              2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|               3.3|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20| 2.566666666666667|
|2014|   11| 12|    2346|  

In [84]:
long_flight2 = df2.filter(df2.distance > 1000 )
long_flight2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    4| 19|    1236|       -4|    1508|       -7|     AS| N309AS|   490|   SEA| SAN|     135|    1050|  12|    36|              2.25|
|2014|   11| 19|    1812|       -3|    2352|       -4|     AS| N564AS|    26|   SEA| ORD|     198|    1721|  18|    12|               3.3|
|2014|    8|  3|    1120|        0|    1415|        2|     AS| N305AS|   656|   SEA| PHX|     154|    1107|  11|    20| 2.566666666666667|
|2014|   11| 12|    2346|  

In [85]:
df2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

In [86]:
selected_1 = df2.select('tailnum', 'origin', 'dest')
selected_1.show()

+-------+------+----+
|tailnum|origin|dest|
+-------+------+----+
| N846VA|   SEA| LAX|
| N559AS|   SEA| HNL|
| N847VA|   SEA| SFO|
| N360SW|   PDX| SJC|
| N612AS|   SEA| BUR|
| N646SW|   PDX| DEN|
| N422WN|   PDX| OAK|
| N361VA|   SEA| SFO|
| N309AS|   SEA| SAN|
| N564AS|   SEA| ORD|
| N323AS|   SEA| LAX|
| N305AS|   SEA| PHX|
| N433AS|   SEA| LAS|
| N765AS|   SEA| ANC|
| N713AS|   SEA| SFO|
| N27205|   PDX| SFO|
| N626AS|   SEA| SMF|
| N8634A|   SEA| MDW|
| N597AS|   SEA| BOS|
| N215AG|   PDX| BUR|
+-------+------+----+
only showing top 20 rows



In [87]:
temp = df2.select(df2.origin,df2.dest,df2.carrier)
temp.show()

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| LAX|     VX|
|   SEA| HNL|     AS|
|   SEA| SFO|     VX|
|   PDX| SJC|     WN|
|   SEA| BUR|     AS|
|   PDX| DEN|     WN|
|   PDX| OAK|     WN|
|   SEA| SFO|     VX|
|   SEA| SAN|     AS|
|   SEA| ORD|     AS|
|   SEA| LAX|     AS|
|   SEA| PHX|     AS|
|   SEA| LAS|     AS|
|   SEA| ANC|     AS|
|   SEA| SFO|     AS|
|   PDX| SFO|     UA|
|   SEA| SMF|     AS|
|   SEA| MDW|     WN|
|   SEA| BOS|     AS|
|   PDX| BUR|     OO|
+------+----+-------+
only showing top 20 rows



In [88]:
filterA = df2.origin == 'SEA'
filterB =df2.dest == 'PDX'


In [89]:
selected_2 = temp.filter(filterA).filter(filterB)
selected_2.show()

+------+----+-------+
|origin|dest|carrier|
+------+----+-------+
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     AS|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     AS|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
|   SEA| PDX|     OO|
+------+----+-------+
only showing top 20 rows



In [90]:
avg_speed = (df2.distance/(df2.air_time/60)).alias("avg_speed")
speed_1 = df2.select('origin','dest','tailnum', avg_speed)
speed_1.show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS|431.29870129870125|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A|481.38888888888886|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

In [91]:
speed_2 =df2.selectExpr('origin','dest','tailnum','distance/(air_time/60) as avg_speed')
speed_2.show()

+------+----+-------+------------------+
|origin|dest|tailnum|         avg_speed|
+------+----+-------+------------------+
|   SEA| LAX| N846VA| 433.6363636363636|
|   SEA| HNL| N559AS| 446.1666666666667|
|   SEA| SFO| N847VA|367.02702702702703|
|   PDX| SJC| N360SW| 411.3253012048193|
|   SEA| BUR| N612AS| 442.6771653543307|
|   PDX| DEN| N646SW|491.40495867768595|
|   PDX| OAK| N422WN|             362.0|
|   SEA| SFO| N361VA| 415.7142857142857|
|   SEA| SAN| N309AS| 466.6666666666667|
|   SEA| ORD| N564AS| 521.5151515151515|
|   SEA| LAX| N323AS| 440.3076923076923|
|   SEA| PHX| N305AS|431.29870129870125|
|   SEA| LAS| N433AS| 409.6062992125984|
|   SEA| ANC| N765AS|474.75409836065575|
|   SEA| SFO| N713AS| 315.8139534883721|
|   PDX| SFO| N27205| 366.6666666666667|
|   SEA| SMF| N626AS|477.63157894736844|
|   SEA| MDW| N8634A|481.38888888888886|
|   SEA| BOS| N597AS| 516.4137931034483|
|   PDX| BUR| N215AG| 441.6216216216216|
+------+----+-------+------------------+
only showing top

In [92]:
df2 = df2.withColumn('distance', df2.distance.cast('float'))
df2 = df2.withColumn('air_time', df2.air_time.cast('float'))

df2.describe('air_time', 'distance').show()

+-------+------------------+-----------------+
|summary|          air_time|         distance|
+-------+------------------+-----------------+
|  count|              9925|            10000|
|   mean|152.88423173803525|        1208.1516|
| stddev|  72.8656286392139|656.8599023464376|
|    min|              20.0|             93.0|
|    max|             409.0|           2724.0|
+-------+------------------+-----------------+



In [93]:
df2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|   132.0|   954.0|   6|    58|               2.2|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|   360.0|  2677.0|  10|    40|               6.0|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|   111.0|   679.0|  14|    43|              1.85|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|    83.0|   569.0|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

In [94]:
df2.filter(df2.origin =='PDX').groupBy().min('distance').show()

+-------------+
|min(distance)|
+-------------+
|        106.0|
+-------------+



In [95]:
df2.filter(df2.origin == 'SEA').groupBy().max('air_time').show()

+-------------+
|max(air_time)|
+-------------+
|        409.0|
+-------------+



In [96]:
df2.filter(df2.carrier == 'DL').filter(df2.origin == 'SEA').groupBy().avg('air_time').show()

+------------------+
|     avg(air_time)|
+------------------+
|188.20689655172413|
+------------------+



In [97]:
df2.withColumn('duration_hrs', df2.air_time/60).groupBy().sum('duration_hrs').show()

+------------------+
| sum(duration_hrs)|
+------------------+
|25289.600000000126|
+------------------+



In [98]:
by_plane = df2.groupBy('tailnum')
by_plane.count().show()

+-------+-----+
|tailnum|count|
+-------+-----+
| N442AS|   38|
| N102UW|    2|
| N36472|    4|
| N38451|    4|
| N73283|    4|
| N513UA|    2|
| N954WN|    5|
| N388DA|    3|
| N567AA|    1|
| N516UA|    2|
| N927DN|    1|
| N8322X|    1|
| N466SW|    1|
|  N6700|    1|
| N607AS|   45|
| N622SW|    4|
| N584AS|   31|
| N914WN|    4|
| N654AW|    2|
| N336NW|    1|
+-------+-----+
only showing top 20 rows



In [99]:
by_origin = df2.groupBy('origin')
by_origin.avg('air_time').show()

+------+------------------+
|origin|     avg(air_time)|
+------+------------------+
|   SEA| 160.4361496051259|
|   PDX|137.11543248288737|
+------+------------------+



In [100]:
df2 = df2.withColumn('dep_delay',df2.dep_delay.cast('float'))
df2.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+------------------+
|2014|   12|  8|     658|     -7.0|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|   132.0|   954.0|   6|    58|               2.2|
|2014|    1| 22|    1040|      5.0|    1505|        5|     AS| N559AS|   851|   SEA| HNL|   360.0|  2677.0|  10|    40|               6.0|
|2014|    3|  9|    1443|     -2.0|    1652|        2|     VX| N847VA|   755|   SEA| SFO|   111.0|   679.0|  14|    43|              1.85|
|2014|    4|  9|    1705|     45.0|    1839|       34|     WN| N360SW|   344|   PDX| SJC|    83.0|   569.0|  17|     5|1.3833333333333333|
|2014|    3|  9|     754|  

In [101]:
by_month_dest = df2.groupBy('month', 'dest')
by_month_dest.avg('dep_delay').show()

+-----+----+--------------------+
|month|dest|      avg(dep_delay)|
+-----+----+--------------------+
|   11| TUS| -2.3333333333333335|
|   11| ANC|   7.529411764705882|
|    1| BUR|               -1.45|
|    1| PDX| -5.6923076923076925|
|    6| SBA|                -2.5|
|    5| LAX|-0.15789473684210525|
|   10| DTW|                 2.6|
|    6| SIT|                -1.0|
|   10| DFW|  18.176470588235293|
|    3| FAI|                -2.2|
|   10| SEA|                -0.8|
|    2| TUS| -0.6666666666666666|
|   12| OGG|  25.181818181818183|
|    9| DFW|   4.066666666666666|
|    5| EWR|               14.25|
|    3| RDM|                -6.2|
|    8| DCA|                 2.6|
|    7| ATL|   4.675675675675675|
|    4| JFK| 0.07142857142857142|
|   10| SNA| -1.1333333333333333|
+-----+----+--------------------+
only showing top 20 rows



In [102]:
from pyspark.sql.types import NumericType
non_numeric_cols = [field.name for field in df2.schema.fields if not isinstance(field.dataType, NumericType)]

In [103]:
print("Non-numeric columns:", non_numeric_cols)

Non-numeric columns: ['year', 'month', 'day', 'dep_time', 'arr_time', 'arr_delay', 'carrier', 'tailnum', 'flight', 'origin', 'dest', 'hour', 'minute']


In [104]:
df= df.withColumnRenamed('faa','dest')
df.show()

+----+--------------------+----------------+-----------------+----+---+---+
|dest|                name|             lat|              lon| alt| tz|dst|
+----+--------------------+----------------+-----------------+----+---+---+
| 04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
| 06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
| 06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
| 06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
| 09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
| 0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
| 0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
| 0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
| 0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
| 0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
| 0W3|Harfor

In [105]:
df3=df2.join(df, on='dest', how='leftouter')
df3.show()

+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+
|dest|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|air_time|distance|hour|minute|      duration_hrs|                name|      lat|        lon| alt| tz|dst|
+----+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+--------+--------+----+------+------------------+--------------------+---------+-----------+----+---+---+
| LAX|2014|   12|  8|     658|     -7.0|     935|       -5|     VX| N846VA|  1780|   SEA|   132.0|   954.0|   6|    58|               2.2|    Los Angeles Intl|33.942536|-118.408075| 126| -8|  A|
| HNL|2014|    1| 22|    1040|      5.0|    1505|        5|     AS| N559AS|   851|   SEA|   360.0|  2677.0|  10|    40|               6.0|       Honolulu Intl|21.318681|-157.922428|  13|-10|  N|
| SFO|2014|    3|  9|    

In [106]:
df4=spark.read.csv("/kaggle/input/datasets-for-pyspark-project/planes.csv",header=True)
df4.show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA

In [107]:
df4=df4.withColumnRenamed('year', 'plane_year')

In [108]:
df5 = df2.join(df4, on='tailnum', how='leftouter')

In [109]:
df5.show()

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|plane_year|                type|  manufacturer|      model|engines|seats|speed|   engine|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+
| N846VA|2014|   12|  8|     658|     -7.0|     935|       -5|     VX|  1780|   SEA| LAX|   132.0|   954.0|   6|    58|               2.2|      2011|Fixed wing multi ...|        AIRBUS|   A320-214|      2|  182|   NA|Turbo-fan|
| N559AS|2014|    1| 22|    1040|      5.0|    1505|        5|     AS|   851|   SEA| HNL

In [110]:
df5.describe().show()

+-------+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-------+-----------------+------+-----+------------------+-----------------+------------------+-----------------+------------------+-----------------+--------------------+--------------------+-------------+--------------------+------------------+-----------------+-------------+
|summary|tailnum|  year|             month|              day|          dep_time|         dep_delay|          arr_time|         arr_delay|carrier|           flight|origin| dest|          air_time|         distance|              hour|           minute|      duration_hrs|       plane_year|                type|        manufacturer|        model|             engines|             seats|            speed|       engine|
+-------+-------+------+------------------+-----------------+------------------+------------------+------------------+------------------+-------+-----------------+-----

In [111]:
df5.show()

+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+
|tailnum|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|flight|origin|dest|air_time|distance|hour|minute|      duration_hrs|plane_year|                type|  manufacturer|      model|engines|seats|speed|   engine|
+-------+----+-----+---+--------+---------+--------+---------+-------+------+------+----+--------+--------+----+------+------------------+----------+--------------------+--------------+-----------+-------+-----+-----+---------+
| N846VA|2014|   12|  8|     658|     -7.0|     935|       -5|     VX|  1780|   SEA| LAX|   132.0|   954.0|   6|    58|               2.2|      2011|Fixed wing multi ...|        AIRBUS|   A320-214|      2|  182|   NA|Turbo-fan|
| N559AS|2014|    1| 22|    1040|      5.0|    1505|        5|     AS|   851|   SEA| HNL

In [112]:
string_value= [field.name for field in df5.schema.fields if not isinstance(field.dataType, NumericType)]

In [113]:
string_value

['tailnum',
 'year',
 'month',
 'day',
 'dep_time',
 'arr_time',
 'arr_delay',
 'carrier',
 'flight',
 'origin',
 'dest',
 'hour',
 'minute',
 'plane_year',
 'type',
 'manufacturer',
 'model',
 'engines',
 'seats',
 'speed',
 'engine']

In [114]:
columns_to_drop = ["year","month","day",'tailnum', 'carrier', 'origin', 'dest', 'type', 'manufacturer', 'model', 'engine']

In [115]:
df5 = df5.drop(*columns_to_drop)

In [116]:
df5.show()

+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|dep_time|dep_delay|arr_time|arr_delay|flight|air_time|distance|hour|minute|      duration_hrs|plane_year|engines|seats|speed|
+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|     658|     -7.0|     935|       -5|  1780|   132.0|   954.0|   6|    58|               2.2|      2011|      2|  182|   NA|
|    1040|      5.0|    1505|        5|   851|   360.0|  2677.0|  10|    40|               6.0|      2006|      2|  149|   NA|
|    1443|     -2.0|    1652|        2|   755|   111.0|   679.0|  14|    43|              1.85|      2011|      2|  182|   NA|
|    1705|     45.0|    1839|       34|   344|    83.0|   569.0|  17|     5|1.3833333333333333|      1992|      2|  149|   NA|
|     754|     -1.0|    1015|        1|   522|   127.0|   937.0|   7|    54|2.1166666666666667|      1999|     

In [117]:
df5.printSchema()

root
 |-- dep_time: string (nullable = true)
 |-- dep_delay: float (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- air_time: float (nullable = true)
 |-- distance: float (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)
 |-- duration_hrs: double (nullable = true)
 |-- plane_year: string (nullable = true)
 |-- engines: string (nullable = true)
 |-- seats: string (nullable = true)
 |-- speed: string (nullable = true)



In [118]:
from pyspark.sql.functions import col,isnan, when, count


In [119]:
from pyspark.sql.functions import mean, col, when

# Define columns to process (only numeric ones)
numeric_columns = ["dep_delay", "air_time", "duration_hrs"]

# Compute mean values for numeric columns
mean_values = df5.select([mean(c).alias(c) for c in numeric_columns]).collect()[0]

# Replace nulls with the mean value for each numeric column
for c in numeric_columns:
    df5 = df5.withColumn(c, when(col(c).isNull(), mean_values[c]).otherwise(col(c)))

df5.show()


+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|dep_time|dep_delay|arr_time|arr_delay|flight|air_time|distance|hour|minute|      duration_hrs|plane_year|engines|seats|speed|
+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|     658|     -7.0|     935|       -5|  1780|   132.0|   954.0|   6|    58|               2.2|      2011|      2|  182|   NA|
|    1040|      5.0|    1505|        5|   851|   360.0|  2677.0|  10|    40|               6.0|      2006|      2|  149|   NA|
|    1443|     -2.0|    1652|        2|   755|   111.0|   679.0|  14|    43|              1.85|      2011|      2|  182|   NA|
|    1705|     45.0|    1839|       34|   344|    83.0|   569.0|  17|     5|1.3833333333333333|      1992|      2|  149|   NA|
|     754|     -1.0|    1015|        1|   522|   127.0|   937.0|   7|    54|2.1166666666666667|      1999|     

In [120]:
string_columns = ["arr_time", "arr_delay", "flight", "hour", "minute", "plane_year", "engines", "seats", "speed"]

df5 = df5.na.fill("unknown", subset=string_columns)


In [121]:
# Define the list of columns where null values need to be replaced
null_col = ["dep_delay", "air_time", "duration_hrs", "plane_year", "engines", "seats", "speed"]

# Replace null values with 0 for the specified columns
df5 = df5.na.fill(value=0, subset=null_col)

# Show the updated DataFrame
df5.show()


+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|dep_time|dep_delay|arr_time|arr_delay|flight|air_time|distance|hour|minute|      duration_hrs|plane_year|engines|seats|speed|
+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|     658|     -7.0|     935|       -5|  1780|   132.0|   954.0|   6|    58|               2.2|      2011|      2|  182|   NA|
|    1040|      5.0|    1505|        5|   851|   360.0|  2677.0|  10|    40|               6.0|      2006|      2|  149|   NA|
|    1443|     -2.0|    1652|        2|   755|   111.0|   679.0|  14|    43|              1.85|      2011|      2|  182|   NA|
|    1705|     45.0|    1839|       34|   344|    83.0|   569.0|  17|     5|1.3833333333333333|      1992|      2|  149|   NA|
|     754|     -1.0|    1015|        1|   522|   127.0|   937.0|   7|    54|2.1166666666666667|      1999|     

In [122]:
df5.printSchema()

root
 |-- dep_time: string (nullable = true)
 |-- dep_delay: double (nullable = false)
 |-- arr_time: string (nullable = false)
 |-- arr_delay: string (nullable = false)
 |-- flight: string (nullable = false)
 |-- air_time: double (nullable = false)
 |-- distance: float (nullable = true)
 |-- hour: string (nullable = false)
 |-- minute: string (nullable = false)
 |-- duration_hrs: double (nullable = false)
 |-- plane_year: string (nullable = false)
 |-- engines: string (nullable = false)
 |-- seats: string (nullable = false)
 |-- speed: string (nullable = false)



In [123]:
from pyspark.sql.functions import col, when

# List of columns to cast to numeric
columns_to_cast = ["dep_time", "arr_time", "arr_delay", "flight", "hour", "minute", "plane_year", "engines", "seats", "speed"]

# Fill non-numeric values in string columns with "0" and cast them to double
for c in columns_to_cast:
    df5 = df5.withColumn(c, when(col(c).isNull(), "0").otherwise(col(c)))  # Replace nulls with "0"
    df5 = df5.withColumn(c, col(c).cast("double"))  # Cast to double

# Cast remaining numeric columns explicitly if needed
numeric_columns = ["dep_delay", "air_time", "distance", "duration_hrs"]
for c in numeric_columns:
    df5 = df5.withColumn(c, col(c).cast("double"))  # Ensure they are double

# Check the schema after conversion
df5.printSchema()

# Show the updated DataFrame
df5.show()


root
 |-- dep_time: double (nullable = true)
 |-- dep_delay: double (nullable = false)
 |-- arr_time: double (nullable = true)
 |-- arr_delay: double (nullable = true)
 |-- flight: double (nullable = true)
 |-- air_time: double (nullable = false)
 |-- distance: double (nullable = true)
 |-- hour: double (nullable = true)
 |-- minute: double (nullable = true)
 |-- duration_hrs: double (nullable = false)
 |-- plane_year: double (nullable = true)
 |-- engines: double (nullable = true)
 |-- seats: double (nullable = true)
 |-- speed: double (nullable = true)

+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|dep_time|dep_delay|arr_time|arr_delay|flight|air_time|distance|hour|minute|      duration_hrs|plane_year|engines|seats|speed|
+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|   658.0|     -7.0|   935.0|     -5.0|1780.0|   132.0|  

In [124]:
# Fill all null values with 0
df5 = df5.na.fill(0)

# Show the DataFrame to verify
df5.show()


+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|dep_time|dep_delay|arr_time|arr_delay|flight|air_time|distance|hour|minute|      duration_hrs|plane_year|engines|seats|speed|
+--------+---------+--------+---------+------+--------+--------+----+------+------------------+----------+-------+-----+-----+
|   658.0|     -7.0|   935.0|     -5.0|1780.0|   132.0|   954.0| 6.0|  58.0|               2.2|    2011.0|    2.0|182.0|  0.0|
|  1040.0|      5.0|  1505.0|      5.0| 851.0|   360.0|  2677.0|10.0|  40.0|               6.0|    2006.0|    2.0|149.0|  0.0|
|  1443.0|     -2.0|  1652.0|      2.0| 755.0|   111.0|   679.0|14.0|  43.0|              1.85|    2011.0|    2.0|182.0|  0.0|
|  1705.0|     45.0|  1839.0|     34.0| 344.0|    83.0|   569.0|17.0|   5.0|1.3833333333333333|    1992.0|    2.0|149.0|  0.0|
|   754.0|     -1.0|  1015.0|      1.0| 522.0|   127.0|   937.0| 7.0|  54.0|2.1166666666666667|    1999.0|    2

In [125]:
from pyspark.sql.functions import col, isnan, when, count

df5.select([count(when(col(c).isNull() | isnan(c), c)).alias(c) for c in df5.columns]).show()


+--------+---------+--------+---------+------+--------+--------+----+------+------------+----------+-------+-----+-----+
|dep_time|dep_delay|arr_time|arr_delay|flight|air_time|distance|hour|minute|duration_hrs|plane_year|engines|seats|speed|
+--------+---------+--------+---------+------+--------+--------+----+------+------------+----------+-------+-----+-----+
|       0|        0|       0|        0|     0|       0|       0|   0|     0|           0|         0|      0|    0|    0|
+--------+---------+--------+---------+------+--------+--------+----+------+------------+----------+-------+-----+-----+



In [126]:
from pyspark.ml.feature import VectorAssembler

In [127]:
features = ["dep_time", "arr_time", "air_time", "distance", "hour", "minute", 
            "duration_hrs", "plane_year", "engines", "seats", "speed"]
target = "dep_delay"

In [128]:
assembler = VectorAssembler(inputCols=features, outputCol="features")
df5 = assembler.transform(df5)

In [129]:
df5 = df5.select("features", target)
df5 = df5.dropna()

In [130]:
df5.show()

+--------------------+---------+
|            features|dep_delay|
+--------------------+---------+
|[658.0,935.0,132....|     -7.0|
|[1040.0,1505.0,36...|      5.0|
|[1443.0,1652.0,11...|     -2.0|
|[1705.0,1839.0,83...|     45.0|
|[754.0,1015.0,127...|     -1.0|
|[1037.0,1352.0,12...|      7.0|
|[847.0,1041.0,90....|     42.0|
|[1655.0,1842.0,98...|     -5.0|
|[1236.0,1508.0,13...|     -4.0|
|[1812.0,2352.0,19...|     -3.0|
|[1653.0,1924.0,13...|     -2.0|
|[1120.0,1415.0,15...|      0.0|
|[811.0,1038.0,127...|     21.0|
|[2346.0,217.0,183...|     -4.0|
|[1314.0,1544.0,12...|     89.0|
|[2009.0,2159.0,90...|      3.0|
|[2015.0,2150.0,76...|     50.0|
|[1017.0,1613.0,21...|     -3.0|
|[2156.0,607.0,290...|     -9.0|
|[1733.0,1945.0,11...|    -12.0|
+--------------------+---------+
only showing top 20 rows



In [131]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

In [132]:
train_data, test_data = df5.randomSplit([0.8, 0.2], seed=42)
print(f"Training data count: {train_data.count()}")
print(f"Testing data count: {test_data.count()}")

Training data count: 8079
Testing data count: 1921


In [133]:
lr = LinearRegression(featuresCol="features", labelCol=target)
lr_model = lr.fit(train_data)

24/11/16 15:10:28 WARN Instrumentation: [3217f761] regParam is zero, which might cause numerical instability and overfitting.
24/11/16 15:10:29 WARN Instrumentation: [3217f761] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [134]:
print(f"Coefficients: {lr_model.coefficients}")
print(f"Intercept: {lr_model.intercept}")

Coefficients: [0.0032643929601843496,7.226373909904947e-05,-0.025561242968149746,0.005765503954030713,0.32844488508922365,-0.02030919874764479,-1.533674578079569,-0.0016460467054503535,-1.146424504489225,0.013908598636287618,-0.044950832444090566]
Intercept: 2.207688125752902


In [135]:
predictions = lr_model.transform(test_data)
predictions.select("features", target, "prediction").show(5)

+--------------------+---------+------------------+
|            features|dep_delay|        prediction|
+--------------------+---------+------------------+
|(11,[0,1,2,3,4,6]...|     -5.0| 6.070459659221246|
|(11,[0,1,2,3,4,6]...|      0.0| 5.257702873458097|
|(11,[0,1,2,3,4,6]...|      0.0| 6.500133514326098|
|(11,[0,1,2,3,4,6]...|     12.0|11.362200336716644|
|(11,[0,1,2,3,5,6]...|     26.0|1.4232006379927689|
+--------------------+---------+------------------+
only showing top 5 rows



In [136]:
evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 28.952721038502585


In [137]:
r2 = evaluator.setMetricName("r2").evaluate(predictions)
print(f"R2 score: {r2}")

R2 score: 0.012652299377983178
